In [1]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

In [12]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
FAISS_INDEX_PATH = "faiss_index"

TEXT_SPLITTER_CHUNK_SIZE = 1000
TEXT_SPLITTER_CHUNK_OVERLAP = 0

HUGGINGFACE_MODELNAME = "google/flan-t5-base"
HUGGINGFACE_MODEL_KWARGS = {"temperature":0, "max_length":512}

LANGCHAIN_QNA_CHAIN_TYPE = "stuff"

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()
db =  FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

d:\InstalledSoftware\anaconda3\envs\LangchainApp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain import HuggingFaceHub
from langchain.chains.question_answering import load_qa_chain

llm = HuggingFaceHub(
    repo_id="google/flan-t5-base"
    , model_kwargs={"temperature":0, "max_length":32})

retriever = db.as_retriever()
output_parser = StrOutputParser()

chain = load_qa_chain(llm, chain_type=LANGCHAIN_QNA_CHAIN_TYPE)

setup_up_retrieval = RunnableParallel(
    {"input_documents": retriever, "question": RunnablePassthrough()}
)

# class LLMInputSetup:
#     def __init__(self) -> None:
#         print("INIT 1 Got Called")

#     def __or__(self, __value: Any) -> UnionType:
#         print("OR 1 Got Called")
#         print(__value)
#         def chained_func(*args, **kwargs):
#             print("CHAIN 1 Got Called")
#             return __value(self.func(*args, **kwargs))
#         print("OR 1 EXIT")
#         return LLMInputSetup(chained_func)
    
#     def __call__(self, *args, **kwargs):
#         print("CALL 1 Got Called")
#         print(args, kwargs)
#         return {"input_documents": args[0], "question": "what did the mayor say ?"}
    

# class LLMTest:
#     def __init__(self) -> None:
#         print("INIT 2 Got Called")
    
#     def __call__(self, *args, **kwargs):
#         print("CALL 2 Got Called")
#         print(args, kwargs)
#         return args[0]
    
# llm_test = LLMTest()
    
# llm_input_setup = LLMInputSetup()

new_chain = setup_up_retrieval | chain

# chain = retriever.__or__(llm_input_setup.__or__(llm_test))

new_chain.invoke("what did the mayor say ?")

{'input_documents': [Document(page_content='We can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. \n\nI’ve worked on these issues a long time. \n\nI know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety.', metadat

In [9]:
query = "what did the mayor say ?"
llm.invoke([{"input_documents": retriever.invoke(query), "question": query}])

ValueError: Message dict must contain 'role' and 'content' keys, got {'input_documents': [Document(page_content='We can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. \n\nI’ve worked on these issues a long time. \n\nI know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll k

In [77]:
chain

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024F0006DC40>)
| HuggingFaceHub(client=<InferenceClient(model='google/flan-t5-base', timeout=None)>, repo_id='google/flan-t5-base', task='text2text-generation', model_kwargs={'temperature': 0, 'max_length': 32})

In [42]:
from langchain import HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="Qiliang/bart-large-cnn-samsum-ChatGPT_v3"
    , model_kwargs={"temperature":0, "max_length":32, 'truncation': 'only_first'})

retriever = db.as_retriever()
output_parser = StrOutputParser()
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser

chain.invoke("what did the mayor say ?")

'The text describes a conversation between two people discussing the recent deaths of two New York City Police Officers and their families. The speakers discuss the'

In [37]:
retriever.invoke("what did the mayor say ?")

[Document(page_content='We can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves. \n\nI’ve worked on these issues a long time. \n\nI know what works: Investing in crime prevention and community police officers who’ll walk the beat, who’ll know the neighborhood, and who can restore trust and safety.', metadata={'source': 'sample

In [34]:
chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024F0006DC40>),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])
| HuggingFaceHub(client=<InferenceClient(model='Qiliang/bart-large-cnn-samsum-ChatGPT_v3', timeout=None)>, repo_id='Qiliang/bart-large-cnn-samsum-ChatGPT_v3', task='text2text-generation', model_kwargs={'temperature': 0, 'max_length': 32, 'truncation': 'only_first'})
| StrOutputParser()

In [36]:
from langchain import HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="Qiliang/bart-large-cnn-samsum-ChatGPT_v3")

output_parser = StrOutputParser()
setup_and_retrieval = RunnableParallel(
    {"query": RunnablePassthrough()}
)

template = """{query}"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | llm | output_parser

chain.invoke("what did the mayor say ?")

ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 792fc9e7-d295-4595-b216-abc0b37764fc)')

In [7]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("{topic}")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"topic": "I am good, how are you"})

'Good'

In [180]:
from types import UnionType
from typing import Any


class Runnable:
    def __init__(self, func, val=1) -> None:
        print("INIT 1 Got Called")
        print(val)
        self.func = func
        self.init_val = 1

    def __or__(self, __value: Any) -> UnionType:
        print("OR 1 Got Called")
        self.init_val += 1
        def chained_func(*args, **kwargs):
            print("CHAIN 1 Got Called")
            return __value(self.func(*args, **kwargs))
        print("OR 1 EXIT")
        return Runnable(chained_func, 2)
    
    def __call__(self, *args, **kwargs):
        print("CALL 1 Got Called")
        return self.func(*args, **kwargs)
    
class Runnable2:
    def __init__(self, func) -> None:
        print("INIT 2 Got Called")
        self.func = func
        self.init_val = 1

    # def __or__(self, __value: Any) -> UnionType:
    #     print("OR 2 Got Called")
    #     self.init_val += 1
    #     def chained_func(*args, **kwargs):
    #         print("CHAIN 2 Got Called")
    #         return __value(self.func(*args, **kwargs))
    #     print("OR 2 EXIT")
    #     return Runnable(chained_func)
    
    def __call__(self, *args, **kwargs):
        print("CALL 2 Got Called")
        return self.func(*args, **kwargs)

In [181]:
def add2(x):
    return x+2

def mul3(x):
    return x*3


add2 = Runnable(add2)
mul3 = Runnable2(mul3)

INIT 1 Got Called
1
INIT 2 Got Called


In [187]:
chain = add2.__or__(mul3)

OR 1 Got Called
OR 1 EXIT
INIT 1 Got Called
2


In [188]:
chain(3)

CALL 1 Got Called
CHAIN 1 Got Called
CALL 2 Got Called


15

In [ ]:
chain = add2 | mul3

OR 1 Got Called
OR 1 EXIT
INIT 1 Got Called
2


## Using LCEL

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

retriever = db.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser

chain.invoke("what did the mayor say ?")

'he said he will always have your back as your President'